<a href="https://colab.research.google.com/github/Armandohaj/Proyecto-Talleretapa/blob/main/lunes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: When **using** this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [19]:
!pip install face_recognition # Install face_recognition package using pip
import face_recognition
import cv2 # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt # pyplot
from pickle import load, dump

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time





In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import cv2
import matplotlib.pyplot as plt

# Define la variable I (asegúrate de darle un valor apropiado)
i = 1
# Ruta de la imagen
img_path = '/content/drive/MyDrive/Colab Notebooks/Images/' + 'img' + str(i) + '.jpg'

# Aquí debes tener una imagen válida en la variable 'img' antes de guardarla
# cv2.imwrite(img_path, img)

def show_image(image_path):
    """ Esta función muestra una imagen.
      Argumentos:
          image_path {str} -- Path de la imagen.
    """
    img = cv2.imread(image_path)
    img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_cvt)
    plt.show()

# Define las coordenadas x1, y1, x2, y2
x1 = 100
y1 = 100
x2 = 200
y2 = 200

# Lista para guardar las rutas de las imágenes
img_list = []

# Agrega la ruta de la imagen a la lista
img_list.append(img_path)

def show_image_with_square(image_path, faces):
    """ Esta función muestra una imagen con un cuadro en cada rostro.
      Argumentos:
          image_path {str} -- Path de la imagen.
          faces {list} -- Lista de diccionarios con las coordenadas de los rostros.
    """
    img = cv2.imread(image_path)
    im2Display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imTemp = im2Display.copy()
    for face in faces:
        x1 = face['x1']
        y1 = face['y1']
        x2 = face['x2']
        y2 = face['y2']
        imTemp = cv2.rectangle(imTemp, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(imTemp)
    plt.show()

# Llama a las funciones según sea necesario
# show_image(img_path)
# face_rectangle = {'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2}
# show_image_with_square(img_path, [face_rectangle])



# Utils

## In-memory data

In [38]:
registered_faces = {}
registered_professors = {}
registered_students = {}
registered_miscellaneous  = []
registered_name_professor = []
lista_estudiantes = []
lista_profesores = []
numero_imagen = 1


## face recognition

In [39]:
def check_if_face_registered(faceencoding):
  for registered_face in registered_faces.items():
    if True in face_recognition.compare_faces([face_encoding], registered_face):
      return True
  return False

## Real time streaming

In [40]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [41]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [46]:
from ipywidgets.widgets import widget
import pickle
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Ingrese su nombre',
    description='Nombre:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Cargar Imagen'
)

or_text = widgets.HTML(
    value=" o "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Registrar Cara')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capturar Imagen')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name_professor = None
registered_name_student = None
registered_miscellaneous = None


# Import necessary libraries (ensure you have them installed)
import numpy as np
import cv2
import face_recognition

# Define a function to handle image upload
def handle_upload(change):
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite('/content/drive/MyDrive/Imagenes/'+ 'nombre'+str(numero_imagen)+'.jpg',image )

# Define a function to handle image capture
def handle_capture(button):
    global uploaded_image_data
    # Start streaming video from the webcam
    video_stream()
    js_reply = video_frame("Capturando...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
        with output:
            clear_output()
            print("Error al capturar la imagen!")

    # Convert JS response to OpenCV Image

    if numero_imagen != numero_imagen:
      numero_imagen+1
    else:
      numero_imagen+=numero_imagen

    uploaded_image_data = js_to_image(js_reply["img"])
    cv2.imwrite('/content/drive/MyDrive/Imagenes/'+ 'nombre'+str(numero_imagen)+'.jpg',uploaded_image_data )

    with output:
        clear_output()
        # Encode the captured frame as PNG
        _, buffer = cv2.imencode('.png', uploaded_image_data)

        # Convert the encoded image to bytes
        image_bytes = buffer.tobytes()
        display(widgets.Image(value=image_bytes))

# Define a function to check if a face is already registered
def check_if_face_registered(encoding):
    global registered_faces
    for face_name, face_encoding in registered_faces.items():
        print(face_encoding)
        matches = face_recognition.compare_faces([face_encoding.get("encoding")], encoding)
        if any(matches):
            return True
    return False

# Define a function to handle face registration
def handle_register(button):
    global registered_faces
    global registered_name_professor
    global registered_name_student
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name}.")

            # Here you can add logic to associate the role with the registered face
            # For example, you can use the role_dropdown value
            # Example: role = role_dropdown.value

            # Store the face encoding along with the role
            registered_faces[registered_name] = {
                'encoding': face_encodings[0],
                'role': role_dropdown.value  # Use the selected role
            }

# Create a dropdown widget for role selection
role_dropdown = widgets.Dropdown(
    options=['Estudiante', 'Profesor', 'Misceláneo'],
    description='Rol:',
    disabled=False
)

# Set up the widget event handlers
upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a button to trigger face registration for professors
register_button_professor = widgets.Button(description='Registrar Profesor')

# Define a function to handle face registration for professors
def handle_register_professor(button):
    global registered_faces
    global registered_name_professor

    if uploaded_image_data is not None:
        # Obtener el nombre del profesor desde el campo de entrada
        registered_name_professor = name_input.value

        # Detectar caras en la imagen cargada
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # Si no se detecta ninguna cara, mostrar una ventana emergente
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # Si se detecta más de una cara, mostrar una ventana emergente
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Obtener las codificaciones de las caras
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Comprobar si la cara ya está registrada
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return


        print(f"Cara registrada para {registered_name_professor} (Profesor).")

        # Recopilar información adicional sobre el profesor

        identificacion = widgets.Text(description = "Identificacion: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        especialidad = widgets.Text(description = "Especialidad: ")
        años_dando_clases = widgets.Text(description = "Años dando clases: " )
        genero = widgets.Text(description = "Genero: ")
        lista_profesores = []
        def guardar_handler(_):
          #Almacenar la codificación de la cara junto con la información del profesor
          profesor ={
              'nombre': registered_name_professor,
              #'encoding': face_encodings[0],
              'role': 'Profesor',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'especialidad': especialidad.value,
                  'años_dando_clases': años_dando_clases.value,
                  'genero': genero.value,
              }
          }

          with open("/content/drive/MyDrive/Infoimagenes/archivoprofesor.txt", "wb") as f:
              pickle.dump(profesor, f)
          lista_profesores.append(profesor)

          registered_faces[registered_name_professor] = profesor
          registered_professors[registered_name_professor] = profesor


          print("La informacion se guardo correctamente")


        guardar_professor_button = widgets.Button(description='Guardar informacion')
        guardar_professor_button.on_click(guardar_handler)
        display(registered_name_professor, identificacion, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, especialidad, años_dando_clases, genero, guardar_professor_button )


register_button_students = widgets.Button(description='Registrar Estudiante')

# Define a function to handle face registration for students
def handle_register_student(button):
    global registered_faces
    global registered_name_student

    if uploaded_image_data is not None:
        registered_name_student = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return


        print(f"Cara registrada para {registered_name_student} (estudiante).")


        #Recopilar información adicional sobre el estudiante
        identificacion = widgets.Text(description = "Identificacion: ")
        nombre = widgets.Text(description = "Nombre: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        carrera = widgets.Text(description = "Carrera: ")
        pasatiempos = widgets.Text (description = "Pasatiempos: ")
        signo_zodiacal = widgets.Text(description = "Signo zodiacal: ")
        lista_estudiantes = []
        def guardar_student(_):

          #Almacenar la codificación de la cara junto con la información del estudiante
          student = {
              'nombre': registered_name_student,
              #'encoding': face_encodings[0],
              'role': 'Student',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'nombre': nombre.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'carrera': carrera.value,
                  'pasatiempos': pasatiempos.value,
                  'signo_zodiacal': signo_zodiacal.value

              }
          }
          with open("/content/drive/MyDrive/Infoimagenes/archivoestudiante.txt", "wb") as f:
              pickle.dump(student, f)
          lista_estudiantes.append(student)



          registered_faces[registered_name_student] = student
          registered_students[registered_name_student] = student


          print("La informacion se guardo correctamente")





        guardar_student_button = widgets.Button(description='Guardar informacion')
        guardar_student_button.on_click(guardar_student)
        display(identificacion, nombre, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, carrera, pasatiempos, signo_zodiacal, guardar_student_button )

register_button_miscellaneo = widgets.Button(description='Registrar Micelanio')
# Define a function to handle face registration for micelaneo
def handle_register_miscellaneous(button):
    global registered_faces
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name_miscellaneous = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name_miscellaneous} (miscellaneous).")

            # Store the face encoding along with the role
            registered_faces[registered_name_miscellaneous] = {
                'encoding': face_encodings[0],
                'role': 'Micelanio'
            }

        # Recopilar información adicional sobre el micelaneo

        identificacion = widgets.Text(description = "Identificacion: ")
        nombre = widgets.Text(description = "Nombre: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        turno = widgets.Text(description = "Turno: ")
        habilidad = widgets.Text(description = "Habilidad: ")
        estado_civil = widgets.Text(description = "estado civil: ")

        def guardar_miscellaneous(_):

          #Almacenar la codificación de la cara junto con la información del micelaneo
          miscelaneo = {
              'nombre': registered_name_miscellaneous,
              #'encoding': face_encodings[0],
              'role': 'Micelaneo',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'nombre': nombre.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'turno': turno.value,
                  'habilidad': habilidad.value,
                  'estado_civil': estado_civil.value
              }
          }
          with open("/content/drive/MyDrive/Infoimagenes/archivomiscelaneo.txt", "wb") as f:
              pickle.dump(miscelaneo, f)

          registered_faces[registered_name_miscellaneous] = miscelaneo
          registered_students[registered_name_miscellaneous] = miscelaneo


          print("La informacion se guardo correctamente")

        guardar_micelaneo_button = widgets.Button(description='Guardar informacion')
        guardar_micelaneo_button.on_click(guardar_miscellaneous)
        display(identificacion, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, turno, habilidad, estado_civil, guardar_micelaneo_button )


# Set up the widget event handler for registering professors
register_button_professor.on_click(handle_register_professor)
register_button_students.on_click(handle_register_student)
register_button_miscellaneo.on_click(handle_register_miscellaneous)
# Create a container for the widgets
container = widgets.VBox([name_input, role_dropdown, widgets.HBox([upload_button, or_text, capture_button]),
                          widgets.HBox([register_button, or_text, register_button_professor, or_text, register_button_students, or_text, register_button_miscellaneo]), output])

# Display the container
display(container)


In [9]:

print(registered_professors)
print(registered_faces)

{}
{}


### informacion del aula

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Lista para almacenar diccionarios de aula
aula_informacion = []

# Crear widgets
numero_de_aula = widgets.Text(description="Número: ")
carnet_miscelaneo = widgets.Text(description="Carnet: ")
capacidad = widgets.Text(description = "Capacidad: ")
aire = widgets.Text(description = "Aire: ")

guardar_button = widgets.Button(description="Guardar Información")
print("informacion del aula: ")

# Función para guardar la información
def guardar_aula(_):
    aula_info = {
        'numero': numero_de_aula.value,
        'carnet': carnet_miscelaneo.value,
        'capacidad': capacidad.value,
        'aire': aire.value
    }
    aula_informacion.append(aula_info)
    print("Información del aula guardada con éxito.")

# Boton para guardar
guardar_button.on_click(guardar_aula)

# Posicion de los widgets
form = widgets.VBox([numero_de_aula, carnet_miscelaneo, capacidad, aire, guardar_button])

# imprimir
display(form)


In [11]:
print(aula_informacion)

[]


### informacion del curso

In [12]:
import ipywidgets as widgets
from IPython.display import display

# diccionario para guardar la informacion del curso
informacion_curso = []
print("informacion del curso: ")

#widgets
codigo_de_curso = widgets.Text(description = "Codigo: ")
nombre_de_curso = widgets.Text(description = "Nombre: ")
periodo = widgets.Text(description = "Periodo: ")
carrera_al_que_pertenece = widgets.Text(description = "Carrera: ")
horario = widgets.Text(description = "Horario: ")
aula = widgets.Text(description = "aula: ")
guardar_button = widgets.Button(description = "Guardar informacion")

# funcion para almacenar la informacion del curso
def guardar_curso(_):
    info_curso = {
      'codigo': codigo_de_curso.value,
      'nombre': nombre_de_curso.value,
      'periodo': periodo.value,
      'carrera': carrera_al_que_pertenece.value,
      'lista_estudiantes' : lista_estudiantes,
      'profesor_lista': lista_profesores,
      'horario': horario.value,
      'aula': aula.value,
    }
    informacion_curso.append(info_curso)
    print("informacion guardada con exito")


def obtener_estudiantes(curso, estudiantes):
    for estudiante, informacion in estudiantes.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None


def obtener_profesor(curso, profesores):
    for profesor, informacion in profesores.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None



# Boton para guardar
guardar_button.on_click(guardar_curso)

# Posicion de los widgets
form = widgets.VBox([codigo_de_curso, nombre_de_curso, periodo, carrera_al_que_pertenece, horario, aula, guardar_button])

# imprimir
display(form)




informacion del curso: 


In [13]:
icape = informacion_curso[0]
print(obtener_profesor(icape, registered_professors))
print(obtener_estudiantes(icape, registered_students))

IndexError: ignored

In [ ]:
print(informacion_curso[0])

### Consultas

In [ ]:
def obtener_profesor(profesores):
  for profesores in profesor():
      print()


# Recognize Faces

In [ ]:
import json
import pickle

# Guardar datos de nuevo en archivos al finalizar el programa
with open("/content/drive/MyDrive/archivoestudiante.txt", "wb") as f:
    pickle.dump(registered_students, f)

with open("/content/drive/MyDrive/archivoprofessor.txt", "wb") as f:
    pickle.dump(registered_professors, f)

with open("/content/drive/MyDrive/archivomicelaneo.txt", "wb") as f:
    pickle.dump(registered_miscellaneous, f)

with open("/content/drive/MyDrive/archivoaula.txt", "wb") as f:
    pickle.dump(aula_informacion, f)


In [ ]:
import cv2


In [ ]:
# Cargar un archivo del drive

with open("/content/drive/MyDrive/archivo1.algo", "rb") as f:
    guardado = load(f)
    print(guardado)

In [ ]:
from datetime import datetime
lista_asistencia = []
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
bbox = ''

while True:
  js_reply = video_frame(label_html, bbox)
  if not js_reply:
    break

  # convert JS response to OpenCV Image
  frame = js_to_image(js_reply["img"])

  # Get the face locations from the frame
  face_locations = face_recognition.face_locations(frame)

  #get the time and date
  current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

  # For each face in the frame
  for (top, right, bottom, left) in face_locations:
      # Draw a rectangle around the face
      cv2.rectangle(
          frame,
          (left, top),
          (right, bottom),
          (0, 0, 255),
          2,
      )
      # Get the face encoding of the face
      face_encoding = face_recognition.face_encodings(frame, [(top, right, bottom, left)])[0]

      # Compare the face encoding to the faces in the database
      matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)
      # variable initialization
      attendance_name = "unknown"
      attendance_path = ""


      # If there is a match
      if True in matches:
          # Get the index of the match
          match_index = matches.index(True)


          # Get the name of the match
          attendance_name = list(registered_faces.keys())[match_index]
      # Save the captured image with the face marked
      captured_image_path = f"captured_images/{current_time}_{attendance_name}.jpg"
      cv2.imwrite(captured_image_path, frame)

      # Append attendance information to the list
      attendance_list.append([attendance_name, captured_image_path, current_time])

    # Show image
      plt.imshow(frame)
      plt.show()

# Print the attendance list
for attendance_info in attendance_list:
    print(f"Name: {attendance_info[0]}, Image Path: {attendance_info[1]}, Time: {attendance_info[2]}")



In [ ]:
import cv2
import face_recognition
import matplotlib.pyplot as plt
import time

# Inicializa la lista de asistencia
attendance_list = []

# Inicializa una base de datos de caras registradas (debes llenar esto con caras y nombres)
registered_faces = {
    "Nombre1": face_recognition.load_image_file("ruta_a_la_imagen1.jpg"),
    "Nombre2": face_recognition.load_image_file("ruta_a_la_imagen2.jpg"),
    # Agrega más caras registradas aquí
}

# Función para capturar el video de la cámara web
def video_stream():
    # Implementa la captura de video desde la cámara web aquí
    pass

# Función para obtener un cuadro de video
def video_frame(label_html, bbox):
    # Implementa la obtención de cuadros de video aquí y retorna un diccionario js_reply con la imagen
    pass

# Función para convertir una respuesta JavaScript en una imagen OpenCV
def js_to_image(js_reply):
    # Implementa la conversión aquí y retorna la imagen en formato OpenCV
    pass

# Inicializa la variable de tiempo actual
current_time = time.strftime("%Y-%m-%d_%H-%M-%S")

# Inicia la captura de video desde la cámara web
video_stream()

# Etiqueta para el video
label_html = 'Capturando...'
bbox = ''

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convierte la respuesta JS a una imagen OpenCV
    frame = js_to_image(js_reply)

    # Obtiene las ubicaciones de las caras en el cuadro
    face_locations = face_recognition.face_locations(frame)

    # Para cada cara en el cuadro
    for face_location in face_locations:
        top, right, bottom, left = face_location

        # Dibuja un rectángulo alrededor de la cara
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Obtiene la codificación de la cara
        face_encoding = face_recognition.face_encodings(frame, [face_location])[0]

        # Compara la codificación de la cara con la base de datos de caras registradas
        matches = face_recognition.compare_faces(list(registered_faces.values()), face_encoding)

        # Si hay una coincidencia
        if True in matches:
            # Obtiene el índice de la coincidencia
            match_index = matches.index(True)

            # Obtiene el nombre de la coincidencia
            match_name = list(registered_faces.keys())[match_index]

            # Guarda la imagen capturada con la cara marcada
            captured_image_path = f"captured_images/{current_time}_{match_name}.jpg"
            cv2.imwrite(captured_image_path, frame)

            # Agrega información de asistencia a la lista
            attendance_list.append([match_name, captured_image_path, current_time])

    # Muestra la imagen
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.show()

# Imprime la lista de asistencia
for attendance_info in attendance_list:
    print(f"Nombre: {attendance_info[0]}, Ruta de la imagen: {attendance_info[1]}, Hora: {attendance_info[2]}")